# Open Access Award
> Scripts to run and adjucate the GSS Open Access Award at SFU



In [136]:
import requests
import pandas as pd
from pathlib import Path
import re
import numpy as np
import random

In [137]:
root = Path("../oaa-2018/data/")
applicants_file = root / "applicants.csv"
review_required_file = root / "applicants_review_required.csv"
reviewed_file = root / "applicants_reviewed.csv"
final_file = root / "applicants_final.csv"
winners_file = root / "winners.csv"
logfile = root / "log.txt"

In [138]:
max_awards = 35
funding = 3500

## Verify applicants

1. Remove any duplicate submissions (same student ID, same DOI)
2. Verify that Summit ID has been provided
3. Verify eligibility of journals (journal lookup on DOAJ via ISSN/name)
4. Mark entries that need manual verification & create CSV

In [139]:
df = pd.read_csv(applicants_file, na_values=[""])

headers = {'Accept': 'application/json'} 
issn_url = "https://doaj.org/api/v1/search/journals/issn:{}"
name_url = "https://doaj.org/api/v1/search/journals/title:{}"

### 1. Duplicate submission

In [140]:
print("*** Remove duplicate submissions")
unique_cols = ['Name (First)', 'Name (Last)', 'Email', 'Student ID#', 'Article Title', 'Please provide the ID of your paper in the SFU Research Summit repository']
df['duplicate_submission'] = df.duplicated(subset=unique_cols)
print("{} duplicate submissions were found among the applications.".format(df['duplicate_submission'].sum()))

*** Remove duplicate submissions
0 duplicate submissions were found among the applications.


### 2. Summit IDs

In [141]:
print("*** Checking summit IDs")
df['uploaded_to_summit'] = df['Please provide the ID of your paper in the SFU Research Summit repository'].notna()
print("{} submission did not provide a Summit ID".format((~df['uploaded_to_summit']).sum()))

*** Checking summit IDs
1 submission did not provide a Summit ID


### 3. Verify eligibility of journals

In [144]:
print("*** Checking journals")
f = open(str(logfile), "w")

i = 1
application_count = len(df)
for ix, row in df[["Journal Name","Journal ISSN"]].iterrows():
    name = row["Journal Name"]
    issn = row["Journal ISSN"]
    
    is_open_access = None
    
    f.write("{}/{} - {} ({})\n".format(i, application_count, name, issn))
    is_issn = re.match("[\S]{4}\-[\S]{4}", issn)
    
    if is_issn:
        r = requests.get(issn_url.format(issn), headers=headers)
        total = r.json()['total']
        f.write("Lookup by ISSN... found {} journals via ISSN\n".format(0))
        if total == 1:
            is_open_access = r.json()['results'][0]['bibjson']['license'][0]['open_access']
        elif total < 1:
            r = requests.get(name_url.format(name), headers=headers)
            total = r.json()['total']
            f.write("ISSN lookup no results. Lookup by name... found {} journals via name\n".format(0))
            if total == 1:
                is_open_access = r.json()['results'][0]['bibjson']['license'][0]['open_access']
    else:
        r = requests.get(name_url.format(name), headers=headers)
        total = r.json()['total']
        f.write("ISSN invalid. Lookup by name... found {} journals via name\n".format(0))
        if total == 1:
            is_open_access = r.json()['results'][0]['bibjson']['license'][0]['open_access']
    
    error_msg = None
    if is_open_access is True:
        f.write("Open Access: Yes\n")
    elif is_open_access is False:
        f.write("Open Access: No\n")
    else:
        error_msg = "Found {} possible candidate journals in DOAJ.\n".format(total)
        f.write("Requiring manual attention: " + error_msg)
        
    df.loc[ix, 'found_on_DOAJ'] = is_open_access
    df.loc[ix, 'doaj_error'] = error_msg
    f.write("\n")
    i = i + 1

f.close()
print("Found in DOAJ: {},requiring attention: {}".format(df['found_on_DOAJ'].sum(), (df['found_on_DOAJ'] != True).sum()))

*** Checking journals
Found in DOAJ: 25,requiring attention: 8


### 4. Create file for manual verification

In [117]:
df['eligible'] = df['found_on_DOAJ'] & df['uploaded_to_summit']
df.loc[df['found_on_DOAJ'].isna(), 'eligible'] = ""
df.to_csv(review_required_file)

## Review & double-check the results

@todo: write up a guideline for reviewing the results

- resources on DOAJ, language around OA

add and fill-in remaining values in the `eligible` column in the file

## Select Winners

In [128]:
eligible = pd.read_csv(reviewed_file, na_values=[""], index_col=0)
eligible['winner'] = False
winners = eligible[eligible.eligible.astype(bool)]

win_ids = []
if winners['Student ID#'].nunique() > max_awards:
    entries = winners['Student ID#'].value_counts()
    while len(win_ids) < max_awards:
        total = entries.sum()
        weights = [x/total for x in entries]
        selected_ids = np.random.choice(entries.keys(), p=weights, size=max_awards-len(win_ids))
        selected_ids = list(set(selected_ids))
        entries.drop(selected_ids, inplace=True)
        win_ids.extend(selected_ids)
else:
    win_ids = winners['Student ID#'].unique()

winners = winners[winners['Student ID#'].isin(win_ids)].drop_duplicates(subset=["Student ID#"])
    
eligible.loc[winners.index, 'winner'] = True
eligible.to_csv(final_file)

print("{} winners | {}$ each. Total: {}$".format(len(winners), funding//len(winners), funding//len(winners)*len(winners)))

27 winners | 129$ each. Total: 3483$


In [151]:
winners = winners[['Name (First)', 'Name (Last)', 'Email', 'Student ID#',
       'Address (Address)', 'Address (Address2)', 'Address (City)',
       'Address (State)', 'Address (Zip)', 'Address (Country)', 'If your application is successful, please indicate the method of delivery for your cheque:']]
winners = winners.reset_index(drop=True)
winners.index = winners.index + 1
winners.to_csv(winners_file)

## Todo

- [ ] Create database of applicants and winners to